In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
from collections import Counter
import os
from argparse import Namespace


flags = Namespace(
    train_file='RAFO.txt',
    seq_size=32,
    batch_size=16,
    embedding_size=64,
    lstm_size=64,
    gradients_norm=5,
    initial_words=['Աշոտը', 'ասաց'],
    predict_top_k=5,
)

In [16]:
def get_data_from_file(train_file, batch_size, seq_size):
    with open(train_file, 'r', encoding='utf-8') as f:
        text = f.read()
    text = text.split()

    word_counts = Counter(text)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {k: w for k, w in enumerate(sorted_vocab)}
    vocab_to_int = {w: k for k, w in int_to_vocab.items()}
    n_vocab = len(int_to_vocab)

    print('Vocabulary size', n_vocab)

    int_text = [vocab_to_int[w] for w in text]
    num_batches = int(len(int_text) / (seq_size * batch_size))
    in_text = int_text[:num_batches * batch_size * seq_size]
    out_text = np.zeros_like(in_text)
    out_text[:-1] = in_text[1:]
    out_text[-1] = in_text[0]
    in_text = np.reshape(in_text, (batch_size, -1))
    out_text = np.reshape(out_text, (batch_size, -1))
    return int_to_vocab, vocab_to_int, n_vocab, in_text, out_text

In [17]:
def get_batches(in_text, out_text, batch_size, seq_size):
    num_batches = np.prod(in_text.shape) // (seq_size * batch_size)
    for i in range(0, num_batches * seq_size, seq_size):
        yield in_text[:, i:i+seq_size], out_text[:, i:i+seq_size]

In [18]:
class RNNModule(nn.Module):
    def __init__(self, n_vocab, seq_size, embedding_size, lstm_size):
        super(RNNModule, self).__init__()
        self.seq_size = seq_size
        self.lstm_size = lstm_size
        self.embedding = nn.Embedding(n_vocab, embedding_size)
        self.lstm = nn.LSTM(embedding_size,
                            lstm_size,
                            batch_first=True)
        self.dense = nn.Linear(lstm_size, n_vocab)
        
    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.dense(output)

        return logits, state

    def zero_state(self, batch_size):
        return (torch.zeros(1, batch_size, self.lstm_size),
                torch.zeros(1, batch_size, self.lstm_size))

def get_loss_and_train_op(net, lr=0.0001):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    return criterion, optimizer

In [29]:
def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    int_to_vocab, vocab_to_int, n_vocab, in_text, out_text = get_data_from_file(
        flags.train_file, flags.batch_size, flags.seq_size)

    net = RNNModule(n_vocab, flags.seq_size,
                    flags.embedding_size, flags.lstm_size)
    
    net.load_state_dict(torch.load('model-23000.pth'))
    
    net.eval()
    
    net = net.to(device)

    criterion, optimizer = get_loss_and_train_op(net, 0.01)

    iteration = 0
    

    for e in range(150):
        batches = get_batches(in_text, out_text, flags.batch_size, flags.seq_size)
        state_h, state_c = net.zero_state(flags.batch_size)
        
        # Transfer data to GPU
        state_h = state_h.to(device)
        state_c = state_c.to(device)
        for x, y in batches:
            iteration += 1
            
            # Tell it we are in training mode
            net.train()

            # Reset all gradients
            optimizer.zero_grad()

            # Transfer data to GPU
            x = torch.tensor(x).to(device)
            y = torch.tensor(y).to(device, dtype=torch.long)

            logits, (state_h, state_c) = net(x, (state_h, state_c))
            loss = criterion(logits.transpose(1, 2), y)

            state_h = state_h.detach()
            state_c = state_c.detach()

            loss_value = loss.item()

            # Perform back-propagation
            loss.backward()

            # Update the network's parameters
            optimizer.step()

            if iteration % 100 == 0:
                print('Epoch: {}/{}'.format(e, 200),
                      'Iteration: {}'.format(iteration),
                      'Loss: {}'.format(loss_value))

            if iteration % 1000== 0:
                predict(device, net, flags.initial_words, n_vocab,
                        vocab_to_int, int_to_vocab, top_k=5)
                torch.save(net.state_dict(),
                           'model-{}.pth'.format(iteration))
                last_name = 'model-{}.pth'.format(iteration)

In [23]:
def predict(device, net, words, n_vocab, vocab_to_int, int_to_vocab, top_k=5):
    net.eval()

    state_h, state_c = net.zero_state(1)
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for w in words:
        ix = torch.tensor([[vocab_to_int[w]]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))
    
    _, top_ix = torch.topk(output[0], k=top_k)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])

    words.append(int_to_vocab[choice])
    

    for _ in range(100):
        ix = torch.tensor([[choice]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))

        _, top_ix = torch.topk(output[0], k=top_k)
        choices = top_ix.tolist()
        choice = np.random.choice(choices[0])
        words.append(int_to_vocab[choice])

    print(' '.join(words))

In [30]:
if __name__ == '__main__':
    main()

Vocabulary size 101000
Epoch: 0/200 Iteration: 100 Loss: 4.782766819000244
Epoch: 0/200 Iteration: 200 Loss: 4.594974994659424
Epoch: 0/200 Iteration: 300 Loss: 4.358212947845459
Epoch: 0/200 Iteration: 400 Loss: 4.35474967956543
Epoch: 0/200 Iteration: 500 Loss: 4.638779163360596
Epoch: 0/200 Iteration: 600 Loss: 4.570209503173828
Epoch: 0/200 Iteration: 700 Loss: 4.694459915161133
Epoch: 0/200 Iteration: 800 Loss: 4.928020000457764
Epoch: 0/200 Iteration: 900 Loss: 4.753615379333496
Epoch: 0/200 Iteration: 1000 Loss: 4.923074722290039
Պատերազմը էր հայկական ամրոցից, որպես նրա դեմ: Ահա, ես իմ հո՛գիս, հպատակները միասին արծաթն կատվի տեղ ծառայել։ Նա ոչինչ չխոսեց, միայն կարողանամ տիրել էր իր հետ զենք, դեպի մի քանի րոպե ևս, վիրավորեց նրա որսորդությունների մեջ։ Այդ միջոցին Մելիք-Փարսադանը և ողջունելով իր «տիկին» խոտերով, առանց զգացմունքի, կարծես գուշակություններ է դնում դեպի զանազան զվարճություններով լեզգու խենջարներ, բայց նա ուներ մի խումբ զանազան օրինակներ։ Այստեղ խառն էր մի շղթա, որ միացն

KeyboardInterrupt: ignored

In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
int_to_vocab, vocab_to_int, n_vocab, in_text, out_text = get_data_from_file(
        flags.train_file, flags.batch_size, flags.seq_size)

Vocabulary size 101000


In [32]:
net = RNNModule(n_vocab, flags.seq_size,
                    flags.embedding_size, flags.lstm_size)
    
net.load_state_dict(torch.load('model-20000.pth'))
    
net.eval()
    
net = net.to(device)

In [87]:
flags = Namespace(
    train_file='RAFO.txt',
    seq_size=32,
    batch_size=16,
    embedding_size=64,
    lstm_size=64,
    gradients_norm=5,
    initial_words=['Հայրը', 'մտածում'],
    predict_top_k=5,
)

In [88]:

predict(device, net, flags.initial_words, n_vocab,
                        vocab_to_int, int_to_vocab, top_k=5)

Հայրը մտածում էր նրան Միքայելը։ «Երբ այդ խոսքը մի այլ բարեգործություն էր, բայց ես չէի հասկացել ես քո խոսքերին քո բոլոր հիմնարկությունները, գուցե քրթմնջոց է քո բոլոր ոչխարները: փա՜ռք և սկսեցի յուր հետ տանելը` մեր մեջ։ Գերանները ևս ճշմարտության համար իրանց անձերը ժամանակները, Անձրևը այն ժամանակ գտնվում էր իր մոտ Այրարատ և մի այլ պետություն — պատասխանեց Մարոն: Երկու տարի օտարազգի և նրա ձայնը կտրեն, միայն կերպարանափոխել նրանով: Բայց այդ դերվիշը դեռ չգիտեին, թե մի օր չէր քարկոծի անապատական ո՞րը Փոքր ազնվականներից միայն հողից կամ քարացած բոցերն և ոչ վարդագույն, ուր էր տարվում Տարբերվում որ իր ձեռքում բռնած ուներ իր անմատչելի լեռներով, իր
